In [1]:
import torch
import os
import numpy as np
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
import cv2
import torchvision
from torchvision import transforms
import albumentations
import albumentations.pytorch

In [20]:
# Dataset
class Dataset(Dataset):
    def __init__(self, transform = None):
        super().__init__()
        
        self.img_dir = "D:/jupyter_proj/AugusTOOTH/classification/data"
        self.filenames = os.listdir(self.img_dir)

        self.transform = transform

    def __getitem__(self, index):
        label = MakeLabel(self.filenames[index])
        img_path = os.path.join(self.img_dir, self.filenames[index])
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # BGR -> RGB
        # albumentations 라이브러리를 사용할 것인데, PIL을 쓰면 변환이 안된다하여 cv2로 변환해준다.
        # img = np.asarray_chkfinite(Image.open(img_path))
        
        if img.dtype == np.uint8:
            img = img / 255.0
        
        img = ToTensor(img)
        
        if self.transform:
            img = self.transform(img)
        #  print(np.shape(img)) # 검사코드
        input = {'data' : img, 'label' : label}
        cnt +=1
        return input
        

    def __len__(self):
        
        return len(self.filenames)

    
def MakeLabel(filenames):
    input_img = filenames
    if "O" in input_img:
        label = F.one_hot(torch.tensor(0), 2)
    else : label = F.one_hot(torch.tensor(1), 2)

    return label

def ToTensor(image):
    image = image.transpose((2,0,1)).astype(np.float32)
    image = torch.from_numpy(image)
    return image

In [21]:
# albumentations train

albumentations_train = albumentations.Compose([
    albumentations.Resize(height=256, width=256),
    albumentations.OneOf([
        albumentations.HorizontalFlip(p=0.8),
        albumentations.RandomRotate90(p=0.8),
        albumentations.VerticalFlip(p=0.8)
    ], p=1),
    albumentations.OneOf([
        albumentations.MotionBlur(p=0.75),
        albumentations.OpticalDistortion(p=0.75),
        albumentations.GaussNoise(p=0.75)
    ], p=1),
    albumentations.pytorch.ToTensorV2()
])

In [22]:
full_dataset = Dataset(transform = albumentations_train)

In [23]:
train_size = int(0.8 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_datset = torch.utils.data.random_split(full_dataset, [train_size, test_size])

In [24]:
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=0)
test_loader = DataLoader(test_datset, batch_size=4, shuffle=False, num_workers=0)

In [25]:
len(train_loader)

135

In [35]:
# model(ResNet50)

import torchvision.models.resnet as resnet
import torch.optim as optim

conv1x1 = resnet.conv1x1
Bottleneck = resnet.Bottleneck
BasicBlock = resnet.BasicBlock

class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=2, zero_init_residual=True):
        super(ResNet, self).__init__()
        self.inplanes = 32
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1, bias=False) # 마찬가지로 전부 사이즈 조정
        self.bn1 = nn.BatchNorm2d(32)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        self.layer1 = self._make_layer(block, 32, layers[0], stride=1) # 3 반복
        self.layer2 = self._make_layer(block, 64, layers[1], stride=2) # 4 반복
        self.layer3 = self._make_layer(block, 128, layers[2], stride=2) # 6 반복
        self.layer4 = self._make_layer(block, 256, layers[3], stride=2) # 3 반복
        
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(256 * block.expansion, num_classes)

        
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)

    def _make_layer(self, block, planes, blocks, stride=1): # planes -> 입력되는 채널 수
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion: 
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        # input [32, 128, 128] -> [C ,H, W]
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        #x.shape =[32, 64, 64]

        x = self.layer1(x)
        #x.shape =[128, 64, 64]
        x = self.layer2(x)
        #x.shape =[256, 32, 32]
        x = self.layer3(x)
        #x.shape =[512, 16, 16]
        x = self.layer4(x)
        #x.shape =[1024, 8, 8]
        
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

In [38]:
resnet50 = ResNet(resnet.Bottleneck, [3,4,6,3], 2, True)

In [39]:
from torchsummary import summary
summary(resnet50, input_size=(3, 128, 128))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 128, 128]             864
       BatchNorm2d-2         [-1, 32, 128, 128]              64
              ReLU-3         [-1, 32, 128, 128]               0
         MaxPool2d-4           [-1, 32, 64, 64]               0
            Conv2d-5           [-1, 32, 64, 64]           1,024
       BatchNorm2d-6           [-1, 32, 64, 64]              64
              ReLU-7           [-1, 32, 64, 64]               0
            Conv2d-8           [-1, 32, 64, 64]           9,216
       BatchNorm2d-9           [-1, 32, 64, 64]              64
             ReLU-10           [-1, 32, 64, 64]               0
           Conv2d-11          [-1, 128, 64, 64]           4,096
      BatchNorm2d-12          [-1, 128, 64, 64]             256
           Conv2d-13          [-1, 128, 64, 64]           4,096
      BatchNorm2d-14          [-1, 128,